---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


def blight_model():    
    train_data = pd.read_csv('train.csv', encoding = "ISO-8859-1", low_memory=False)
    test_data = pd.read_csv('test.csv')   
    address = pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')    
    
    # Clean train set and test_set  ----------------------------------------------------------------------  
    # NaN
    train_data = train_data.replace(np.nan,'0')
    test_data = test_data.replace(np.nan,'0')
    
    # Lower case
    columns = ['agency_name','inspector_name','violator_name','mailing_address_str_name','city','state','non_us_str_code','country','disposition']
    for c in columns:
        train_data[c] = train_data[c].apply(lambda x: x.lower())
        test_data[c]  = test_data[c].apply(lambda x: x.lower())
    address.address   = address.address.apply(lambda x: x.lower())
    latlons.address  = latlons.address.apply(lambda x: x.lower())    
    
    # Delete columns
    columns_to_delete = ['agency_name','violator_name','ticket_issued_date','hearing_date']
    test_data.drop(columns_to_delete, axis='columns', inplace=True)
    
    columns_to_delete = ['violation_street_number','violation_street_name','violation_zip_code','violation_description']
    test_data.drop(columns_to_delete, axis='columns', inplace=True)
    
    columns_to_delete = ['fine_amount','admin_fee','state_fee','late_fee','clean_up_cost','grafitti_status']
    test_data.drop(columns_to_delete, axis='columns', inplace=True)
    
    columns_to_delete = ['mailing_address_str_number','mailing_address_str_name','city','state','non_us_str_code']
    test_data.drop(columns_to_delete, axis='columns', inplace=True)
    
    # Address where the violation occurred -> lat lon 
    train_data = pd.merge(train_data,address, left_on='ticket_id', right_on='ticket_id')
    train_data = pd.merge(train_data,latlons, left_on='address', right_on='address')   
    test_data = pd.merge(test_data,address, left_on='ticket_id', right_on='ticket_id')
    test_data = pd.merge(test_data,latlons, left_on='address', right_on='address')   
    train_data.drop('address', axis='columns', inplace=True)
    test_data.drop('address', axis='columns', inplace=True)    

    # Delete: Not responsible
    train_data = train_data[train_data.compliance.notnull()]    
    Y_train_data = train_data.compliance
    
    # Keep only columns in test_data
    columnas = test_data.columns
    X_train_data = train_data[columnas].copy()    

    zip_code       = list(set(list(train_data['zip_code']) + list(test_data['zip_code'])))
    inspector_name = list(set(list(train_data['inspector_name']) + list(test_data['inspector_name'])))
    #violator_name  = list(set(list(train_data['violator_name'])  + list(test_data['violator_name'])))
    country        = list(set(list(train_data['country'])        + list(test_data['country'])))    
    violation_code = list(set(list(train_data['violation_code']) + list(test_data['violation_code'])))    
    disposition    = list(set(list(train_data['disposition'])    + list(test_data['disposition'])))   
    
#    COLUMNS      = [inspector_name,violator_name,country,violation_code,disposition]
#    COLUMNS_NAME = ['inspector_name','violator_name','country','violation_code','disposition']
    COLUMNS      = [inspector_name,zip_code,country,violation_code,disposition]
    COLUMNS_NAME = ['inspector_name','zip_code','country','violation_code','disposition'] 
   
    for j,C in enumerate(COLUMNS):
        d = dict(zip(C,np.arange(len(C))))
        X_train_data[COLUMNS_NAME[j]].replace(d,inplace=True)      
        test_data[COLUMNS_NAME[j]].replace(d,inplace=True)    
        
    X_train_data.lat.replace(np.nan,0,inplace=True) 
    X_train_data.lon.replace(np.nan,0,inplace=True) 
    Y_train_data.replace({'0':0,'1':1},inplace=True) 
    test_data.lat.replace(np.nan,0,inplace=True) 
    test_data.lon.replace(np.nan,0,inplace=True) 
   
    
    # Split (train and test) ----------------------------------------------------------------------------
    X_train,X_test,y_train,y_test = train_test_split(X_train_data,Y_train_data,random_state=0)#,test_size=0.8
    
    # Model ---------------------------------------------------------------------------------------------
    model = RandomForestClassifier()
    model.fit(X_train,y_train)
    
#    print('Accuracy of RF classifier on training set: {:.2f}'.format(model.score(X_train, y_train)))
#    print('Accuracy of RF classifier on test set: {:.2f}'.format(model.score(X_test, y_test)))
    
    
    PREDICTION = model.predict_proba(test_data)
    
    out = test_data.set_index('ticket_id')
    out['compliance'] = PREDICTION.T[1]
    
    return out['compliance'] #[train_data,test_data]

In [11]:
blight_model()


ticket_id
284932    0.0
285362    0.0
285361    0.3
285338    0.0
285346    0.3
285345    0.0
285347    0.2
285342    0.7
285530    0.2
284989    0.1
285344    0.0
285343    0.0
285340    0.0
285341    0.0
289828    0.0
289830    0.5
289829    0.2
292133    0.2
292134    0.0
285349    0.3
285348    0.0
284991    0.1
285532    0.2
286073    0.2
285406    0.0
285001    0.5
285006    0.1
365862    1.0
285405    0.0
287857    0.0
         ... 
376276    0.2
376218    0.1
376368    0.5
376369    0.4
376225    0.0
376222    0.0
376362    0.6
376363    0.6
376228    0.4
376265    0.0
376286    0.4
376320    0.1
376314    0.0
376327    0.3
376435    0.3
376434    0.1
376459    0.2
376478    0.1
376473    0.2
376484    0.0
376482    0.1
376480    0.1
376479    0.1
376481    0.1
376483    0.0
376496    0.1
376497    0.1
376499    0.1
376500    0.2
369851    0.7
Name: compliance, Length: 61001, dtype: float64